0. 내가 가진 사진을 이용해서 dataset을 만드는 법(lab-10_4_1)
1. 배운 내용을 이용해서 학습하기
2. 모델을 저장하고 다시 불러오는 법

# 10-4 ImageFolder (2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/My Drive/수업/Info_Sys_2021/PyTorch

/content/drive/My Drive/수업/Info_Sys_2021/PyTorch


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

In [ ]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(1, 120),       #잘 모르는 경우
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        #out = self.layer3(out)
        return out
        

In [ ]:
net = CNN().to(device)

### 방법 1

In [ ]:
pip install pytorch_model_summary

In [ ]:
import pytorch_model_summary
print(pytorch_model_summary.summary(net, torch.zeros(1, 3, 64, 128).to(device), show_input=False))

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv2d-1     [1, 6, 60, 124]             456             456
            ReLU-2     [1, 6, 60, 124]               0               0
       MaxPool2d-3      [1, 6, 30, 62]               0               0
          Conv2d-4     [1, 16, 26, 58]           2,416           2,416
            ReLU-5     [1, 16, 26, 58]               0               0
       MaxPool2d-6     [1, 16, 13, 29]               0               0
Total params: 2,872
Trainable params: 2,872
Non-trainable params: 0
-----------------------------------------------------------------------


layer2의 output 16\*13\*29 가 layer3의 nn.Linear 함수의 input이 된다.

## 방법 2

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(1, 120),       #잘 모르는 경우
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        print(out.shape)
        out = self.layer2(out)
        print(out.shape)
        out = out.view(out.shape[0], -1)
        print(out.shape)
        #out = self.layer3(out)
        return out

In [ ]:
net = CNN().to(device)

In [ ]:
#testing 
test_input = (torch.Tensor(1,3,64,128)).to(device)
test_out = net(test_input)

torch.Size([1, 6, 30, 62])
torch.Size([1, 16, 13, 29])
torch.Size([1, 6032])


layer2의 output 16\*13\*29 (=6032) 가 layer3의 nn.Linear 함수의 input이 된다.

Network 구성을 완료하자.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [ ]:
net = CNN().to(device)

In [ ]:
print(pytorch_model_summary.summary(net, torch.zeros(1, 3, 64, 128).to(device), show_input=False))

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv2d-1     [1, 6, 60, 124]             456             456
            ReLU-2     [1, 6, 60, 124]               0               0
       MaxPool2d-3      [1, 6, 30, 62]               0               0
          Conv2d-4     [1, 16, 26, 58]           2,416           2,416
            ReLU-5     [1, 16, 26, 58]               0               0
       MaxPool2d-6     [1, 16, 13, 29]               0               0
          Linear-7            [1, 120]         723,960         723,960
            ReLU-8            [1, 120]               0               0
          Linear-9              [1, 2]             242             242
Total params: 727,074
Trainable params: 727,074
Non-trainable params: 0
-----------------------------------------------------------------------


In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
total_batch = len(data_loader)

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')   

[Epoch:1] cost = 0.6687480211257935
[Epoch:2] cost = 0.6291600465774536
[Epoch:3] cost = 0.5118286609649658
[Epoch:4] cost = 0.2979542315006256
[Epoch:5] cost = 0.12197210639715195
[Epoch:6] cost = 0.05593332648277283
[Epoch:7] cost = 0.026890290901064873
Learning Finished!


훈련된 모델을 저장하자.

먼저, 현재 폴더 아래에 model이라는 폴더를 만들어 놓는다.

A common PyTorch convention is to save models using either a .pt or .pth file extension.

In [ ]:
# Specify a path
PATH = "model/entire_model.pt"

# Save
torch.save(net, PATH)


In [ ]:
# Load
model = torch.load(PATH).to(device)

Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.

In [ ]:
model.eval()

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=6032, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=2, bias=True)
  )
)

In [ ]:
print(net.layer1[0])
print(model.layer1[0])


Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))


In [ ]:
print(net.layer1[0].weight[0][0][0])
print(model.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == model.layer1[0].weight[0]

tensor([-0.0920,  0.0741,  0.1044, -0.0159,  0.0489], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.0920,  0.0741,  0.1044, -0.0159,  0.0489], device='cuda:0',
       grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

test  data를 사용하자.

In [ ]:
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
